In [1]:
import pandas as pd
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
amd_file = "test_data/test_amd.csv"
stock = pd.read_csv(amd_file)
stock_df = pd.DataFrame(stock)
stock_df.fillna(0)

,High-Low Price,Close-Open Price,7-day m_avg,14-day m_avg,21-day m_avg,7-day m_std,close
0,0.629999,0.010000,0.629999,0.629999,0.629999,0.000000,11.430000
1,0.280001,-0.020000,0.455000,0.455000,0.455000,0.000000,11.430000
2,0.460000,-0.190001,0.456667,0.456667,0.456667,0.109697,11.240000
3,0.380000,0.030000,0.437500,0.437500,0.437500,0.092557,11.320000
4,0.330000,0.120000,0.416000,0.416000,0.416000,0.100350,11.490000
...,...,...,...,...,...,...,...
1002,1.200005,0.009995,2.527143,2.815714,3.009048,2.298816,91.809998
1003,2.320000,-1.330002,2.600001,2.836429,3.009048,2.138821,91.599998
1004,3.029999,-1.040001,2.794286,2.850000,3.075238,1.749579,90.620003
1005,2.659996,1.510002,2.585714,2.644286,2.906190,0.934162,92.290001


In [4]:
X = stock_df

In [5]:
X_train, X_test = train_test_split(X)

In [6]:
# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=6, activation="relu", input_dim=6))

# Add our second Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=3, activation="relu"))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [8]:
# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

NameError: name 'y_train' is not defined

In [9]:
# Plot the accuracy
history_df.plot(y="accuracy")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

NameError: name 'history_df' is not defined